In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation
import time
import pandas as pd
from itertools import count
from IPython import display

import seaborn as sns
from IPython.display import HTML
from scipy.io import loadmat
from operator_functions.pointer_func import pointer_vector
from operator_functions.Laplace_operator import LaplaceOpt_Aq,uBC_Laplace,b_vector_creator
from operator_functions.Conjugate_gradient import CG_solver
from operator_functions.bound_append import bound_appender

In [5]:
##diffusivity
alpha  = 0.005 ;
## Delta t
dt = 0.02 ;
## grid
Nx=31
Ny=40
xtotal  = np.linspace(0,1,Nx+2)
x=xtotal[1:Nx+1]
ytotal  = np.linspace(0,1,Ny+2)
y=ytotal[1:Ny+1]
dx = x[1] - x[0] 
dy=y[1] - y[0] 
xv, yv = np.meshgrid(x, y, indexing='ij')

X,Y=np.meshgrid(xtotal, ytotal, indexing='ij')
kappa=0.02*np.exp(-((((xv-0.7)**2)/0.09)+(((yv-0.6)**2)/0.25)))
b=kappa/(alpha)
Q=np.zeros((Nx*Ny,1))


BCB=(-0.3)*np.ones(Nx)
BCL=0.5-0.5*np.cos(2*np.pi*y)
BCT=0.5+0.5*np.sin(4*np.pi*x-0.5*np.pi)
BCR=np.zeros(Ny)
Solution_Matrix=CG_solver(Q,b,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR)
BCBt=(Solution_Matrix[:,0]-1*dx*(-1*0.3))
BCLt=0.5-0.5*np.cos(2*np.pi*ytotal)
BCTt=0.5+0.5*np.sin((4*np.pi*xtotal)-(0.5*np.pi))
BCRt=Solution_Matrix[-1,:]

Final_matrix=bound_appender(Solution_Matrix,Nx, Ny,BCBt,BCLt,BCTt,BCRt)

fig,ax=plt.subplots(1,1)
cp = ax.contourf(X, Y, Final_matrix)
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()


In [4]:
Final_matrix

array([[0.        , 0.02025351, 0.07937323, 0.17256963, 0.29229249,
        0.42884258, 0.57115742, 0.70770751, 0.82743037, 0.92062677,
        0.97974649, 1.        , 0.97974649, 0.92062677, 0.82743037,
        0.70770751, 0.57115742, 0.42884258, 0.29229249, 0.17256963,
        0.07937323, 0.02025351, 0.        ],
       [0.23461078, 0.22097442, 0.25227265, 0.31516334, 0.40065841,
        0.50016119, 0.60475896, 0.70547489, 0.79381864, 0.86240143,
        0.90550387, 0.91952883, 0.90329398, 0.85813649, 0.78781769,
        0.6982279 , 0.59689664, 0.4922998 , 0.39289937, 0.3056857 ,
        0.2335209 , 0.16917452, 0.07937323],
       [0.39037642, 0.37674006, 0.3935545 , 0.43512273, 0.49498146,
        0.5663442 , 0.64219624, 0.71556294, 0.7799113 , 0.82959527,
        0.86027372, 0.86924947, 0.85569418, 0.8207368 , 0.7674056 ,
        0.70042086, 0.62583513, 0.55049779, 0.481261  , 0.42369932,
        0.37980184, 0.34350762, 0.29229249],
       [0.5156521 , 0.50201573, 0.51003504, 0.536

In [ ]:
fig,ax=plt.subplots(1,1)
cp = ax.contourf(xv, yv, Solution_Matrix)
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
fig,ax=plt.subplots(1,1)
cp = ax.contourf(xv, yv, b/(1))
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
fig,ax=plt.subplots(1,1)
cp = ax.contourf(xv, yv, kappa/(1))
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
b.shape

In [ ]:
Nx=4
Ny=3
N=Nx*Ny
xtotal  = np.linspace(0,1,Nx+2)
x=xtotal[1:Nx+1]
ytotal  = np.linspace(0,1,Ny+2)
y=ytotal[1:Ny+1]
dx = x[1] - x[0] 
dy=y[1] - y[0] 

In [ ]:
matrix=[]
for i in range(N):
    Q=np.zeros((N,1))
    Q[i,0]=1
    matrix.append(list(LaplaceOpt_Aq(Q,Nx,Ny,dx,dy)[:,0]))
matrx_data=pd.DataFrame(np.array(matrix))
matrx_data.columns=[i for i in matrx_data.columns]
matrx_data.index=[i for i in matrx_data.index]
inverted_matrix=np.linalg.inv(np.array(matrix))
matrix_inverse=pd.DataFrame(inverted_matrix)
matrix_inverse.columns=[i for i in matrix_inverse.columns]
matrix_inverse.index=[i for i in matrix_inverse.index]



In [ ]:
matrx_data

In [ ]:
inverted_vector=inverted_matrix@(b_vector_creator(b,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR))
inverted_vector_Matrix=np.zeros((Nx,Ny))
k=0
for i in range(Nx):
    for j in range(Ny):
        inverted_vector_Matrix[i,j]=inverted_vector[k]
        k=k+1
fig,ax=plt.subplots(1,1)
cp = ax.contourf(xv, yv, inverted_vector_Matrix)
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
2/(dy**2)

In [ ]:
iQ=pointer_vector(Nx, Ny)
pointer_mapping_visual=pd.DataFrame(pointer_vector(Nx, Ny))
pointer_mapping_visual.columns=[f"x={i}" for i in pointer_mapping_visual.columns]
pointer_mapping_visual.index=[f"y={len(pointer_mapping_visual.index)-(i+1)}" for i in pointer_mapping_visual.index]

In [ ]:
pointer_mapping_visual

In [ ]:
pointer_mapping_actual=pd.DataFrame((pointer_vector(Nx, Ny)).T)
pointer_mapping_actual.columns=[f"i={i}" for i in pointer_mapping_actual.columns]
pointer_mapping_actual.index=[f"j={i}" for i in pointer_mapping_actual.index]

In [ ]:
pointer_mapping_actual

In [ ]:
fig,ax=plt.subplots(1,1)
cp = ax.contourf(xv, yv, Solution_Matrix)
fig.colorbar(cp)
#ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
q=pd.DataFrame(np.flipud((Solution_Matrix.T)))
q.columns=[f"i={i+1}" for i in q.columns]
q.index=[f"j={len(q.index)-i}" for i in q.index]

In [ ]:
uBC_Vector=uBC_Laplace(Nx,Ny,dx,dy,BCB,BCL,BCT,BCR)
uBC_Vector_Matrix=np.zeros((Nx,Ny))
k=0
for i in range(Nx):
    for j in range(Ny):
        uBC_Vector_Matrix[i,j]=uBC_Vector[k]
        k=k+1



In [ ]:
pd.DataFrame(np.flipud(uBC_Vector_Matrix.T))

In [ ]:
np.sqrt(0.05)

In [ ]:
1/(20)

In [ ]:
BCL

In [ ]:
BCR

In [ ]:
BCB

In [ ]:
BCT